# API_자치구단위(내국인)

In [1]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
import datetime, calendar, timedelta, time
from collections import OrderedDict
import urllib
import re

In [2]:
# Date list
year = int(input('Insert year ex)2020'))
month = input('Insert month ex)04, 12')
month_int = int(month)

lastday = calendar.monthrange(year, month_int)[1]
dt_list = list()

for day in range(1, lastday+1):
    dt = datetime.date(year,month_int,day)
    dt = dt.strftime("%Y%m%d")
    dt_list.append(dt)

len(dt_list)

Insert year ex)2020 2020
Insert month ex)04, 12 05


31

In [3]:
# Setting Variable
key = '***'
start_idx = 1
end_idx = 100
d_date = dt_list[0]
url = 'http://openapi.seoul.go.kr:8088/{}/json/SPOP_LOCAL_RESD_JACHI/1/100/{}'.format(key, d_date)

# Response to dict
response = urllib.request.urlopen(url)
json_str = response.read()
json_object = json.loads(json_str)

# Total count
list_total_count = json_object['SPOP_LOCAL_RESD_JACHI']['list_total_count']
list_total_count

600

In [4]:
url

'http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_JACHI/1/100/20200501'

In [5]:
# url_list
url_list = list()

for day in dt_list:
    d_date = str(day)
    start_idx = 1
    end_idx = 100
    
    for i in range(0,int(list_total_count/end_idx)+1):
        if end_idx <= list_total_count:
            url = 'http://openapi.seoul.go.kr:8088/{}/json/SPOP_LOCAL_RESD_JACHI/{}/{}/{}'.format(key, start_idx, end_idx, d_date)
            url_list.append(url)
        else:
            end_idx = list_total_count
            url = 'http://openapi.seoul.go.kr:8088/{}/json/SPOP_LOCAL_RESD_JACHI/{}/{}/{}'.format(key, start_idx, end_idx, d_date)
            url_list.append(url)
            break            
        start_idx += 100
        end_idx += 100             
            
len(url_list)

217

In [6]:
url_list[-5:]

['http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_JACHI/201/300/20200531',
 'http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_JACHI/301/400/20200531',
 'http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_JACHI/401/500/20200531',
 'http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_JACHI/501/600/20200531',
 'http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_JACHI/601/600/20200531']

## Extract Data

In [7]:
# url_list

In [9]:
df_raw_list = list()

for url_i in url_list:
    response = urllib.request.urlopen(url_i)
    json_str = response.read()
    json_object = json.loads(json_str)

    #Extract dict
    dict_raw = json_object['SPOP_LOCAL_RESD_JACHI']['row']
    #dict to df
    df_raw = pd.DataFrame(dict_raw)
    df_raw_list.append(df_raw)
    #print(url_i)

len(df_raw_list)

KeyError: 'SPOP_LOCAL_RESD_JACHI'

In [10]:
df_raw = pd.concat(df_raw_list, axis=0).reset_index(drop=True)
print(df_raw.shape)
df_raw.head()

(600, 32)


,STDR_DE_ID,TMZON_PD_SE,ADSTRD_CODE_SE,TOT_LVPOP_CO,MALE_F0T9_LVPOP_CO,MALE_F10T14_LVPOP_CO,MALE_F15T19_LVPOP_CO,MALE_F20T24_LVPOP_CO,MALE_F25T29_LVPOP_CO,MALE_F30T34_LVPOP_CO,...,FEMALE_F25T29_LVPOP_CO,FEMALE_F30T34_LVPOP_CO,FEMALE_F35T39_LVPOP_CO,FEMALE_F40T44_LVPOP_CO,FEMALE_F45T49_LVPOP_CO,FEMALE_F50T54_LVPOP_CO,FEMALE_F55T59_LVPOP_CO,FEMALE_F60T64_LVPOP_CO,FEMALE_F65T69_LVPOP_CO,FEMALE_F70T74_LVPOP_CO
0,20200501,00,11110,207572.7085,4862.9403,2838.9254,4896.0967,9616.2358,10208.4014,7823.3271,...,8227.0112,6934.3864,7732.1275,7339.2481,8801.0352,7461.6656,8202.2744,7119.5769,5461.9432,15445.9707
1,20200501,00,11140,172324.0726,3777.5903,1519.1125,2592.1535,6263.8586,8050.3246,8701.6942,...,8665.3305,8029.665,8855.8119,6579.7364,6885.3175,5748.1929,6489.8277,5673.3853,4149.0118,10679.3262
2,20200501,00,11170,256468.2971,6798.0081,3395.4596,5783.9243,8675.3334,11641.25,11992.9908,...,12384.9772,11826.709,12472.309,9838.5169,10600.9414,8456.5985,9200.1933,8269.6105,5845.8238,17649.0584
3,20200501,00,11200,311460.8074,11342.6493,4824.7448,7166.1294,10575.2063,12633.8645,12320.9059,...,13125.8124,13252.551,14800.5888,11434.5035,12294.9133,10800.1467,12238.7131,10757.7469,7975.4695,20690.1658
4,20200501,00,11215,371362.061,11139.2732,6093.9766,10155.9146,15459.026,18036.9422,15581.7246,...,18318.9464,16131.0561,15184.1215,13583.9532,14901.5184,13540.7611,14196.7886,12727.4606,8797.7612,21139.237


In [ ]:
df_raw['ADSTRD_CODE_SE'].unique()

### Unload

In [ ]:
# Unload by file type
path_out = 'C:/Users/sky/DataAnalytics/result_table/'
file_name = '서울시열린데이터광장_생활인구_자치구단위(내국인)_{}{}'.format(year, month)
file_type = ['.txt', '.xlsx']

for type in file_type:
    if type == '.txt':
        path = path_out+file_name+type
        df_raw.to_csv(path, sep='\t', index=False, encoding='UTF8')
    elif type == '.xlsx':
        path = path_out+file_name+type
        df_raw.to_excel(path, index=False)
    else:
        print('Wrong file type')

# path_out + file_name + file_type[0]
# df_raw.to_csv('C:/Users/sky/DataAnalytics/result_table/서울시열린데이터광장_생활인구_자치구단위(내국인)_202004.txt', sep='\t', index=False, encoding='UTF8')
# df_raw.to_excel('C:/Users/sky/DataAnalytics/result_table/서울시열린데이터광장_생활인구_자치구단위(내국인)_202004.xlsx', index=False)